In [3]:
import pandas as pd
import sqlite3
import sys
import os
import numpy as np
from pathlib import Path
from dotenv import load_dotenv
load_dotenv()

from data.snowflake_pull import get_snowflake_config, setconnection, run_query_to_df
import data.sql_lite_store as sql_lite_store
import data.snowflake_pull as snowflake_pull

import data.demand_pull as demand_pull
import agents.workingFlow as workingFlow

In [2]:
def find_project_root(start=None, markers=("pyproject.toml", "requirements.txt", ".git", ".env")):
    p = Path(start or os.getcwd()).resolve()
    for parent in (p, *p.parents):
        if any((parent / m).exists() for m in markers):
            return parent
    return Path.cwd().resolve()

ROOT = find_project_root()
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

LOCAL_DB_PATH = ROOT / "data" / "inventory_data.db"
LOCAL_DB_PATH.parent.mkdir(parents=True, exist_ok=True)

In [4]:
df_demand = sql_lite_store.load_table("demand_data")
df_kepplerSplits = sql_lite_store.load_table("Keppler_Split_Perc")
df_vendor_cbm = sql_lite_store.load_table("Vendor_CBM")
df_CBM_Max = sql_lite_store.load_table("CBM_Max")

config = get_snowflake_config()
conn = setconnection(config)

df_skuSupplySnapshot = snowflake_pull.run_query_to_df(conn, snowflake_pull.SQL_SKU_Supply_Snapshot)

In [16]:
df_skuSupplySnapshot.columns

Index(['MC1', 'MC2', 'BRAND', 'PSKU', 'SKU', 'PRODUCT_NAME', 'PUBBED', 'OH',
       'T90_DAILY_AVG', 'F90_DAILY_AVG', 'AVG_LT', 'OO', 'NEXT_DELIVERY',
       'T90_DOS_OH', 'F90_DOS_OH', 'F90_DOS_OO', 'T90_BELOW', 'F90_BELOW',
       'ALERT'],
      dtype='object')

In [12]:
print(df_demand.columns)

Index(['product_merch_classification2', 'parent_product_part_number',
       'product_part_number', 'product_name', 'Code', 'vendor_name',
       'vendor_purchaser_code', 'Purchase MOQ', 'Master Case Pack',
       'Case Pk CBM', 'Final Buy Qty', 'Vendor Earliest ETD',
       'upload_timestamp'],
      dtype='object')


0       139518
1       139519
2       162803
3       248629
4       608214
        ...   
283    2085542
284    2085606
285    2029310
286    2031726
287    2032726
Name: product_part_number, Length: 288, dtype: int64

In [133]:
def keep_first_max_avglt(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df["AVG_LT"] = pd.to_numeric(df["AVG_LT"], errors="coerce")
    idx = df.groupby("SKU")["AVG_LT"].idxmax()   # index of max AVG_LT per SKU
    return df.loc[idx].reset_index(drop=True)

In [134]:
#combine the 2 tables,
df_demand['product_part_number'] = df_demand['product_part_number'].astype(str).str.strip()
df_skuSupplySnapshot['SKU']= df_skuSupplySnapshot['SKU'].astype(str).str.strip()
#detected duplicated SKU in df_skuSupplySnapshot, keep the first max AVG_LT
df_skuSupplySnapshot = keep_first_max_avglt(df_skuSupplySnapshot)


df_sku_data = df_demand.merge(
    df_skuSupplySnapshot,
    how='left',
    left_on='product_part_number',
    right_on='SKU'
)

# Drop requested columns (only if present)
cols_to_drop = ['product_merch_classification2', 'PRODUCT_NAME', 'PSKU', 'ALERT', 'upload_timestamp']
df_sku_data = df_sku_data.drop(columns=[c for c in cols_to_drop if c in df_sku_data.columns])

df_sku_data = df_sku_data.rename(columns={'OO': 'ost_ord', "Code": "vendor_Code", "Purchase MOQ": "MOQ", "Master Case Pack": "MCP", "Final Buy Qty": "planned_demand"})
df_sku_data = df_sku_data.rename(columns={'Case Pk CBM': 'case_pk_CBM', 'Vendor Earliest ETD': 'vendor_earliest_ETD', 'NEXT_DELIVERY': 'Next_Delivery'})

avg_lt_mean = df_sku_data['AVG_LT'].mean(skipna=True)
num_cols = ["AVG_LT", "F90_DAILY_AVG", "OH", "ost_ord", "planned_demand"]
df_sku_data[num_cols] = df_sku_data[num_cols].apply(pd.to_numeric, errors="coerce").astype(float)
df_sku_data['AVG_LT'] = df_sku_data['AVG_LT'].fillna(avg_lt_mean)
df_sku_data['ost_ord'] = df_sku_data['ost_ord'].fillna(0)
df_sku_data['OH'] = df_sku_data['OH'].fillna(0)

In [135]:
df_sku_data["baseConsumption"] = np.where(
    df_sku_data["F90_DAILY_AVG"].notna(),
    (df_sku_data["AVG_LT"] + 4 * 7) * df_sku_data["F90_DAILY_AVG"],
    0.0
)

df_sku_data["bufferConsumption"] = np.where(
    df_sku_data["F90_DAILY_AVG"].notna(),
    (df_sku_data["AVG_LT"] + 8 * 7) * df_sku_data["F90_DAILY_AVG"],
    df_sku_data["planned_demand"]
)

base_qty = df_sku_data["baseConsumption"] - (df_sku_data["OH"].fillna(0) + df_sku_data["ost_ord"].fillna(0))
buffer_qty = df_sku_data["bufferConsumption"] - (df_sku_data["OH"].fillna(0) + df_sku_data["ost_ord"].fillna(0))

df_sku_data["baseDemand"] = np.maximum(base_qty, 0)
df_sku_data["bufferDemand"] = np.maximum(buffer_qty, 0)
df_sku_data["baseDemand"] = np.minimum(df_sku_data["baseDemand"], df_sku_data["planned_demand"].fillna(0))
df_sku_data["excess_demand"] = np.maximum(df_sku_data["planned_demand"].fillna(0) - df_sku_data["bufferDemand"], 0)

df_sku_data["baseDemand"] = np.where(
    df_sku_data["baseConsumption"] == 0,
    df_sku_data["planned_demand"],             # if baseConsumption == 0 → use planned_demand
    df_sku_data["baseDemand"]                  # otherwise keep existing baseDemand
)

#snapping baseDemand to MOQ, since its not really a choice
df_sku_data["baseDemand"] = np.maximum(df_sku_data["baseDemand"], df_sku_data["MOQ"])

#snapping baseDemand to the higher of mcp multiples
m = df_sku_data["MCP"]
df_sku_data["baseDemand"] = np.ceil(df_sku_data["baseDemand"] / m) * m
df_sku_data["excess_demand"] = np.floor(df_sku_data["excess_demand"] / m) * m




In [35]:
df_sku_data.to_csv("sku_data_check.csv", index=False)

In [122]:
df_sku_data.columns

Index(['parent_product_part_number', 'product_part_number', 'product_name',
       'vendor_Code', 'vendor_name', 'vendor_purchaser_code', 'MOQ', 'MCP',
       'case_pk_CBM', 'planned_demand', 'vendor_earliest_ETD', 'MC1', 'MC2',
       'BRAND', 'SKU', 'PUBBED', 'OH', 'T90_DAILY_AVG', 'F90_DAILY_AVG',
       'AVG_LT', 'ost_ord', 'Next_Delivery', 'T90_DOS_OH', 'F90_DOS_OH',
       'F90_DOS_OO', 'T90_BELOW', 'F90_BELOW', 'baseConsumption',
       'bufferConsumption', 'baseDemand', 'bufferDemand', 'excess_demand'],
      dtype='object')

In [37]:
print(df_kepplerSplits.columns)
print(df_vendor_cbm.columns)
print(df_CBM_Max.columns)


Index(['ITEM_ID', 'CARTONIZATION_FLAG', 'TOTAL_STAT_FCAST', 'TLA1_FCAST',
       'TNY1_FCAST', 'MDT1_FCAST', 'TLA1_FRAC', 'TNY1_FRAC', 'MDT1_FRAC'],
      dtype='object')
Index(['PRODUCT', 'CHW_SKU_NUMBER', 'MC1_NAME', 'MC2_NAME', 'MC3_NAME',
       'BRAND', 'CUSTOMER_EARLIEST_TARGET_DATE', 'EARLIEST_TARGET_DATE',
       'CHW_MOQ_LEVEL', 'CHW_OTB', 'CHW_PRIMARY_SUPPLIER_NAME',
       'CHW_PRIMARY_SUPPLIER_NUMBER', 'CHW_MASTER_CASE_PACK',
       'CHW_MASTER_CARTON_CBM'],
      dtype='object')
Index(['vendor_number', 'vendor_name', 'CBM Max', 'Unnamed: 3',
       'upload_timestamp'],
      dtype='object')


In [143]:
df_CBM_Max.columns


Index(['vendor_number', 'vendor_name', 'CBM Max', 'Unnamed: 3',
       'upload_timestamp'],
      dtype='object')

In [136]:
import importlib, states.state_loader
importlib.reload(states.state_loader)
sku_data_state_list = states.state_loader.df_to_chewy_sku_states(df_sku_data)

In [ ]:
import importlib, states.state_loader
importlib.reload(states.state_loader)
vendor_state_list = states.state_loader.df_to_vendor_states(df_sku_data, df_CBM_Max, sku_data_state_list)

KeyError: 'vendor_number'

In [148]:
df_sku_data.head()

,parent_product_part_number,product_part_number,product_name,vendor_Code,vendor_name,vendor_purchaser_code,MOQ,MCP,case_pk_CBM,planned_demand,...,T90_DOS_OH,F90_DOS_OH,F90_DOS_OO,T90_BELOW,F90_BELOW,baseConsumption,bufferConsumption,baseDemand,bufferDemand,excess_demand
0,166463.0,139518,"Frisco Birthday Cake Dog & Cat Hat, X-Small/Small",3755,"HANGZHOU TIANYUAN PET PRODUCTS CO., LTD.",JADESCHACK,1500.0,50,0.052896,1000.0,...,NaN,NaN,NaN,NaN,NaN,0.00,1000.00,1500.0,1000.00,0.0
1,166463.0,139519,"Frisco Birthday Cake Dog & Cat Hat, Medium/Large",3755,"HANGZHOU TIANYUAN PET PRODUCTS CO., LTD.",JADESCHACK,1500.0,25,0.070756,3108.0,...,NaN,NaN,NaN,NaN,NaN,0.00,3108.00,3125.0,3108.00,0.0
2,189514.0,162803,"Frisco Dog & Cat Birthday Bandana, Medium/Large",3755,"HANGZHOU TIANYUAN PET PRODUCTS CO., LTD.",JADESCHACK,1500.0,150,0.019575,5806.0,...,300.19,80.35,65.35,False,True,15298.50,18062.10,1500.0,3681.10,2100.0
3,189514.0,248629,"Frisco Dog & Cat Birthday Bandana, X-Small/Small",3755,"HANGZHOU TIANYUAN PET PRODUCTS CO., LTD.",JADESCHACK,1500.0,250,0.024000,1277.0,...,259.36,44.08,253.89,False,True,2289.35,2702.91,1500.0,0.00,1250.0
4,608198.0,608214,"Frisco Dog & Cat Birthday Bandana, Pink, Mediu...",3755,"HANGZHOU TIANYUAN PET PRODUCTS CO., LTD.",JADESCHACK,1500.0,144,0.024000,1105.0,...,441.70,125.96,40.41,False,True,4419.05,5217.33,1584.0,474.33,576.0
